In [23]:
import os, sys
import numpy as np
from math import *
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf

In [29]:
data_dir = 'tmp_datasets/'

feature_description = {'group_id': tf.io.FixedLenFeature([], tf.int64),
                        'timestep': tf.io.FixedLenFeature([], tf.int64),
                        'parameter_vector': tf.io.FixedLenSequenceFeature([], tf.float32,allow_missing=True),
                        'pos': tf.io.FixedLenFeature([], tf.string),
                        'vel': tf.io.FixedLenFeature([], tf.string)}


In [26]:
records_dataset =  tf.data.TFRecordDataset(tf.data.Dataset.list_files([data_dir + filename for filename in os.listdir(data_dir)]))


In [33]:
def _parse_record(x):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(x, feature_description)

parsed_records_dataset = records_dataset.map(_parse_record)

def _parse_tensor(x):
    output = {'group_id': x['group_id'],
                'timestep': x['timestep'],
                'parameter_vector': x['parameter_vector'],
                'pos': tf.io.parse_tensor(x['pos'],out_type=tf.float32),
                'vel': tf.io.parse_tensor(x['vel'],out_type=tf.float32)}
    return output

parsed_tensors_dataset = parsed_records_dataset.map(_parse_tensor)


In [34]:
for d in tqdm(parsed_tensors_dataset):
    print(d)
    break


0it [00:00, ?it/s]

{'group_id': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'timestep': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'parameter_vector': <tf.Tensor: shape=(7,), dtype=float32, numpy=
array([0.9      , 0.       , 6.632653 , 1.       , 5.       , 6.2831855,
       0.1      ], dtype=float32)>, 'pos': <tf.Tensor: shape=(100, 2), dtype=float32, numpy=
array([[  7.8362103 ,  81.2605    ],
       [139.0096    ,  69.9703    ],
       [130.86319   , 126.14565   ],
       [ 72.93689   ,  59.899673  ],
       [  9.3802805 ,  84.558136  ],
       [122.98649   , 147.9031    ],
       [ 67.27479   , 134.91426   ],
       [169.66393   , 105.66742   ],
       [134.49442   ,  70.66713   ],
       [117.96186   , 149.16856   ],
       [165.49422   , 113.09389   ],
       [ 76.70161   ,  26.074215  ],
       [ 10.270233  ,  92.204956  ],
       [  4.3742023 ,  79.58971   ],
       [  0.9938342 ,  79.826965  ],
       [  4.8785014 ,  82.75725   ],
       [126.37269   , 147.69844   ],
       [  7.205348  ,  8

In [19]:
    print(tf.io.parse_tensor(d['pos'],out_type=tf.float32))

NameError: name 'f' is not defined

In [20]:
#records_dataset = tf.data.TFRecordDataset(record_file)

# Create a dictionary describing the features.



parsed_records_dataset


<MapDataset shapes: {group_id: (), parameter_vector: (None,), pos: (), timestep: (), vel: ()}, types: {group_id: tf.int64, parameter_vector: tf.float32, pos: tf.string, timestep: tf.int64, vel: tf.string}>

In [24]:
for d in tqdm(parsed_records_dataset):
    pass#print(tf.io.parse_tensor(d['pos'],out_type=tf.float32))

25000it [00:02, 9430.56it/s]
